In [ ]:
import numpy as np
import pandas as pd
import transformers
from gliner import GLiNER
import warnings

warnings.filterwarnings("ignore")

model = GLiNER.from_pretrained("urchade/gliner_large-v2.1")
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-cased')
df=pd.read_csv('/app/Name_Screening/Susovan/NER_TRY.csv')
#df=df.iloc[0:5,:]
# List of all possible labels
labels = ['Primary Name', 'Alias Name', 'Place Of Birth', 'Nationality', 'Previous Name',
          'Person Identity card Number', 'TypeofIdentityCard', 'DateofBirth', 'Year of Birth', 'Gender', 
          'Maritial Status', 'Report date', ' Identity Card Expiry Date', 'Identity Card Issue Date', 
          'Age', 'Age reported date', 'Married reported date', 'Organization Name', 
          'Occupation_Position', 'Shareholding percentage', 'Sector', 'Organization Location','Associate Name','Sister Name','Sister-in-law name','Mother-in-law Name','Spouse Name','Daughter Name','Father Name','Son Name','Brother Name','Daughter-in-law name','Granddaughter Name','Grandmother Name','Grandson Name','Mother Name','Brother-In-law Name','Son-in-law Name','Grandfather Name','Deceased','Deceased Date','Location','crime','crime location','crime amount','crime date','Action Taken for crime','Native of','Resident of','Title like general']

# Define max chunk size and overlap size
max_chunk_size = 320
overlap_size = 100

def predict_chunk_labels(chunk):
    chunk_text = tokenizer.convert_tokens_to_string(chunk)  # Convert chunk tokens back to text
    predictions = model.predict_entities(chunk_text, labels=labels)
    return predictions

def process_biography(text):
    # Tokenization and chunking
    tokens = tokenizer.tokenize(text)
    chunks = []
    for i in range(0, len(tokens), max_chunk_size - overlap_size):
        chunk = tokens[i:i + max_chunk_size]
        chunks.append(chunk)
    
    # Predict labels for each chunk and store them
    all_predicted_labels = []
    for chunk in chunks:
        chunk_labels = predict_chunk_labels(chunk)
        all_predicted_labels.extend(chunk_labels)
    
    # Organize the entities in a dictionary
    entities_dict = {"text": text}
    for label in labels:
        entities_dict[label] = []

    for entity in all_predicted_labels:
        label = entity['label']
        value = entity['text']
        entities_dict[label].append(value)
    
    # Convert list values to a single string with comma separation or NaN if empty
    for label in entities_dict:
        if label != "text":
            if entities_dict[label]:
                entities_dict[label] = ",, ".join(entities_dict[label])
            else:
                entities_dict[label] = np.nan
    
    return entities_dict

# Apply the process to each biography in the DataFrame
processed_biographies = df['V_BIOGRAPHY'].apply(lambda x: process_biography(x))

# Convert list of dictionaries to a DataFrame
processed_df = pd.DataFrame(processed_biographies.tolist())

# Merge the processed results with the original DataFrame
final_df = df.drop(columns=['V_BIOGRAPHY']).join(processed_df)

# Display the resulting DataFrame
final_df.to_excel('/app/Name_Screening/Susovan/Gliner_result2.xlsx',index=False)
